In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from litequeue import SQLQueue

import sqlite3

In [ ]:
TEST_1 = "key_test_1"
TEST_2 = "key_test_2"

In [ ]:
for conn_type_kwargs in (
    {"filename_or_conn": sqlite3.connect(":memory:")},
    {"filename_or_conn": ":memory:"},
    {"memory": True},
):
    q = SQLQueue(**conn_type_kwargs)
    assert (
        q.conn.isolation_level is None
    ), f"Isolation level not set properly for connection '{conn_type_kwargs}'"

In [ ]:
q = SQLQueue(sqlite3.connect(":memory:"))

q.put("hello")
q.put("world")
q.put("foo")
q.put("bar")

4

In [ ]:
q.pop()

{'message': 'hello', 'message_id': '4098d9bda6a8f71cd085ecc28797675e'}

In [ ]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7fae25cee730>, items=[{'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'hello',
  'message_id': '4098d9bda6a8f71cd085ecc28797675e',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': None,
  'message': 'world',
  'message_id': '39889bf2a5360878c0e9f786d88f38f7',
  'status': 0},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': None,
  'message': 'foo',
  'message_id': '1ee51cfbbf0c877e652e1bf052c27a08',
  'status': 0},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': None,
  'message': 'bar',
  'message_id': 'b1f150a87f683427f5223b37545c285d',
  'status': 0}])


In [ ]:
# pop remaining
for _ in range(3):
    q.pop()


assert q.pop() is None

In [ ]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7fae25cee730>, items=[{'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'hello',
  'message_id': '4098d9bda6a8f71cd085ecc28797675e',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'world',
  'message_id': '39889bf2a5360878c0e9f786d88f38f7',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'foo',
  'message_id': '1ee51cfbbf0c877e652e1bf052c27a08',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'bar',
  'message_id': 'b1f150a87f683427f5223b37545c285d',
  'status': 1}])


In [ ]:
q.put("hello")
q.put("world")
q.put("foo")
q.put("bar")

8

In [ ]:
task = q.pop()

assert task["message"] == "hello"

In [ ]:
q.peek()

{'message': 'world',
 'message_id': '5192c5908bf9db658075698375ceae20',
 'status': 0,
 'in_time': 1615838997,
 'lock_time': None,
 'done_time': None}

In [ ]:
# next one that is free
assert q.peek()["message"] == "world"

# status = 0 = free
assert q.peek()["status"] == 0

In [ ]:
task["message"], task["message_id"]

('hello', '8c60f8e06f40ec89b628475ac5460d9d')

In [ ]:
q.done(task["message_id"])

8

In [ ]:
q.get(task["message_id"])

{'message': 'hello',
 'message_id': '8c60f8e06f40ec89b628475ac5460d9d',
 'status': 2,
 'in_time': 1615838997,
 'lock_time': 1615838997,
 'done_time': 1615838997}

In [ ]:
already_done = q.get(task["message_id"])

# stauts = 2 = done
assert already_done["status"] == 2

in_time = already_done["in_time"]
lock_time = already_done["lock_time"]
done_time = already_done["done_time"]

assert done_time >= lock_time >= in_time
print(
    f"Task {already_done['message_id']} took {done_time - lock_time} seconds to get done and was in the queue for {done_time - in_time} seconds"
)

Task 8c60f8e06f40ec89b628475ac5460d9d took 0 seconds to get done and was in the queue for 0 seconds


In [ ]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7fae25cee730>, items=[{'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'hello',
  'message_id': '4098d9bda6a8f71cd085ecc28797675e',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'world',
  'message_id': '39889bf2a5360878c0e9f786d88f38f7',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'foo',
  'message_id': '1ee51cfbbf0c877e652e1bf052c27a08',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'bar',
  'message_id': 'b1f150a87f683427f5223b37545c285d',
  'status': 1},
 {'done_time': 1615838997,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'hello',
  'message_id': '8c60f8e06f40ec89b628475ac5460d9d',
  'status': 2},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': None,
  'message': 'world',
  'message_id': '5192

In [ ]:
assert q.qsize() == 7

In [ ]:
next_one_msg = q.peek()["message"]
next_one_id = q.peek()["message_id"]

task = q.pop()

assert task["message"] == next_one_msg
assert task["message_id"] == next_one_id

In [ ]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7fae25cee730>, items=[{'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'hello',
  'message_id': '4098d9bda6a8f71cd085ecc28797675e',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'world',
  'message_id': '39889bf2a5360878c0e9f786d88f38f7',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'foo',
  'message_id': '1ee51cfbbf0c877e652e1bf052c27a08',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'bar',
  'message_id': 'b1f150a87f683427f5223b37545c285d',
  'status': 1},
 {'done_time': 1615838997,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'hello',
  'message_id': '8c60f8e06f40ec89b628475ac5460d9d',
  'status': 2},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838998,
  'message': 'world',
  'message_id':

In [ ]:
q.prune()

In [ ]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7fae25cee730>, items=[{'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'hello',
  'message_id': '4098d9bda6a8f71cd085ecc28797675e',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'world',
  'message_id': '39889bf2a5360878c0e9f786d88f38f7',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'foo',
  'message_id': '1ee51cfbbf0c877e652e1bf052c27a08',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838997,
  'message': 'bar',
  'message_id': 'b1f150a87f683427f5223b37545c285d',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': 1615838998,
  'message': 'world',
  'message_id': '5192c5908bf9db658075698375ceae20',
  'status': 1},
 {'done_time': None,
  'in_time': 1615838997,
  'lock_time': None,
  'message': 'foo',
  'message_id': '725d32ef93c3

In [ ]:
from string import ascii_lowercase, printable
from random import choice


def random_string(string_length=10, fuzz=False, space=False):
    """Generate a random string of fixed length """
    letters = ascii_lowercase
    letters = letters + " " if space else letters
    if fuzz:
        letters = printable
    return "".join(choice(letters) for i in range(string_length))

In [ ]:
q = SQLQueue(":memory:", maxsize=50)

In [ ]:
for i in range(50):

    q.put(random_string(20))

In [ ]:
assert q.qsize() == 50

Make sure an error is raised when the queue has reached its size limit

In [ ]:
import sqlite3

try:
    q.put(random_string(20))
except sqlite3.IntegrityError:  # max len reached
    assert q.full() == True
    print("test pass")

test pass


In [ ]:
q.pop()

{'message': 'xjrzkgoorjcmsrbzuyaa',
 'message_id': '05eecf3a55df7325195faa9a4110bcec'}

In [ ]:
assert q.full() == False

In [ ]:
q.put("hello")

51

In [ ]:
q.empty()

False

In [ ]:
assert q.empty() == False

q2 = SQLQueue(":memory:")

assert q2.empty() == True

**Random benchmarks**

Inserting 100 items in the queue. Since it will run many times, the queue will end up having a lot more than 100 items

In [ ]:
import gc

In-memory SQL queue

In [ ]:
q = SQLQueue(":memory:", maxsize=None)

In [ ]:
gc.collect()

264

In [ ]:
%%timeit -n10000 -r7

q.put(random_string(20))

41.6 µs ± 4.79 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
q.qsize()

70000

Standard python queue.

In [ ]:
from queue import Queue

In [ ]:
q = Queue()

In [ ]:
gc.collect()

113

In [ ]:
%%timeit -n10000 -r7

q.put(random_string(20))

26.9 µs ± 7.43 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Persistent SQL queue

In [ ]:
q = SQLQueue("test.queue", maxsize=None)

In [ ]:
gc.collect()

69

In [ ]:
%%timeit -n10000 -r7

q.put(random_string(20))

172 µs ± 13.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
!du -sh test.queue*

9.1M	test.queue
32K	test.queue-shm
5.0M	test.queue-wal


In [ ]:
!rm test.queue*

In [ ]:
assert q.conn.isolation_level is None

Creating and removing tasks

In [ ]:
q = Queue()

In [ ]:
gc.collect()

135

In [ ]:
%%timeit -n10000 -r7

tid = random_string(20)

q.put(tid)

q.get()

q.task_done()

31.9 µs ± 3.28 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
q = SQLQueue(":memory:", maxsize=None)

In [ ]:
gc.collect()

69

In [ ]:
%%timeit -n10000 -r7

tid = random_string(20)

q.put(tid)

task = q.pop()

q.done(task["message_id"])

99.8 µs ± 6.94 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
